In [ ]:
%load_ext autoreload
%autoreload 2

Please keep this notebook functionally at par with scripts/oncall/forecast.py AND linearly runnable (no code with duplicated functionality; run all from this cell down should produce a report).

In [ ]:
import numpy as np
import pandas as pd

import pickle
import argparse
import copy
import json
import time

import sys
sys.path.append('../../')

from main.seir.fitting import calculate_loss, train_val_split
from main.seir.forecast import create_decile_csv
from utils.create_report import create_report
from utils.enums import Columns
from viz import plot_trials, plot_r0_multipliers
from main.seir.uncertainty import get_all_ptiles, forecast_ptiles
from main.seir.uncertainty import save_r0_mul, predict_r0_multipliers


In [ ]:
folder = 'test/mumbai47'
district = 'mumbai'
iterations = 1000

In [ ]:
with open(f'../../reports/{folder}/predictions_dict.pkl', 'rb') as pkl:
    region_dict = pickle.load(pkl)

In [ ]:
forecast_dict = {}

In [ ]:
if district == 'pune':
    date_of_interest = '2020-06-15'
    # df_reported = region_dict['m2']['df_district']
    df_reported = region_dict['m2']['df_district_unsmoothed']
elif district == 'mumbai':
    date_of_interest = '2020-06-30'
    df_reported = pd.read_csv(f'../../reports/{folder}/true.csv')
    df_reported['date'] = pd.to_datetime(df_reported['date'])
    # df_reported = region_dict['m2']['df_district_unsmoothed']

In [ ]:
deciles_idx, forecast_dict['beta'] = get_all_ptiles(region_dict, date_of_interest, iterations)
# with open(f'../../reports/{folder}/deciles_idx.json', 'w+') as params_json:
#     deciles_idx['beta'] = forecast_dict['beta']
#     json.dump(deciles_idx, params_json, indent=4)
#     del deciles_idx['beta']

In [ ]:
# forecast deciles to csv
forecast_dict['decile_params'], deciles_forecast = forecast_ptiles(region_dict, deciles_idx)
df_output = create_decile_csv(deciles_forecast, df_reported, region_dict['dist'], 'district', icu_fraction=0.02)
# df_output.to_csv(f'../../reports/{folder}/deciles.csv')
# pd.DataFrame(forecast_dict['decile_params']).to_csv(f'../../reports/{folder}/deciles-params.csv')

In [ ]:
idxs = list(deciles_idx.values())
predictions = region_dict['m2']['predictions']
params = region_dict['m2']['params']
losses = region_dict['m2']['losses']
def get_idxs(l, idxs):
    return [l[i] for i in idxs]
plots = plot_trials(region_dict, 
    k=len(deciles_idx), predictions=get_idxs(predictions,idxs), 
    params=get_idxs(params,idxs), losses=get_idxs(losses,idxs), 
    plot_individual_curves=True, vline=date_of_interest, which_compartments=[Columns.active])
# plots[Columns.active].figure.savefig(f'../../reports/{folder}/deciles.png')
forecast_dict['deciles_plot'] = plots[Columns.active]

In [ ]:

# perform what-ifs on 80th percentile
predictions_mul_dict = predict_r0_multipliers(region_dict, params[deciles_idx[80]])
save_r0_mul(predictions_mul_dict, folder=folder)

forecast_dict['what-ifs-plot'] = plot_r0_multipliers(region_dict, params[deciles_idx[80]], predictions_mul_dict, multipliers=[0.9, 1, 1.1, 1.25])
# forecast_dict['what-ifs-plot'].figure.savefig(f'../../reports/{folder}/what-ifs/what-ifs.png')

In [ ]:
# create_report(region_dict, forecast_dict=forecast_dict, ROOT_DIR=f'../../reports/{folder}')